# Scraping data from steam market

In [4]:
# Team and Tournament names
tournament_names = [
    "2018 FACEIT London", "2018 ELEAGUE Boston", "2017 PGL Krakow",
    "2017 ELEAGUE Atlanta", "2016 MLG Columbus", "2016 ESL One Cologne",
    "2015 ESL One Cologne", "2015 DreamHack Cluj-Napoca",
    "2015 ESL One Katowice", "2014 ESL One Cologne"
]

tournaments = {}
tournaments["London 2018"] = [
    "compLexity Gaming", "Team Liquid", "Astralis", "Natus Vincere", "BIG",
    "FaZe Clan", "HellRaisers", "MIBR", "Ninjas in Pyjamas", "Fnatic",
    "G2 Esports", "Vega Squadron", "Cloud9", "TYLOO", "Winstrike Team",
    "mousesports"
]
tournaments["Boston 2018"] = [
    "Cloud9", "G2 Esports", "FaZe Clan", "Vega Squadron", "Space Soldiers",
    "mousesports", "Natus Vincere", "Quantum Bellator Fire", "Team Liquid",
    "Renegades", "AVANGAR", "Sprout", "Team Envy", "Misfits Gaming",
    "Flash Gaming", "FlipSid3 Tactics"
]
tournaments["Krakow 2017"] = [
    "Gambit Esports", "BIG", "SK Gaming", "North", "Astralis", "Virtus.pro",
    "Immortals", "Fnatic", "Cloud9", "FlipSid3 Tactics", "G2 Esports",
    "Natus Vincere", "mousesports", "PENTA Sports", "FaZe Clan",
    "Vega Squadron"
]
tournaments["Atlanta 2017"] = [
    "Gambit Esports", "Team Liquid", "SK Gaming", "North", "Astralis", "Virtus.pro",
    "GODSENT", "Fnatic", "OpTic Gaming", "FlipSid3 Tactics", "G2 Esports",
    "Natus Vincere", "mousesports", "Team EnVyUs", "FaZe Clan", "HellRaisers"
]
tournaments["Columbus 2016"] = [
    "Ninjas in Pyjamas", "G2 Esports", "Splyce", "Cloud9", "Team EnVyUs", "Virtus.pro",
    "Astralis", "Fnatic", "Counter Logic Gaming", "Team Liquid", "FaZe Clan",
    "Natus Vincere", "Gambit Gaming", "FlipSid3 Tactics", "mousesports", "Luminosity Gaming"
]
tournaments["Cologne 2016"] = [
    "Ninjas in Pyjamas", "G2 Esports", "OpTic Gaming", "Team Dignitas", "Team EnVyUs", "Virtus.pro",
    "Astralis", "Fnatic", "Counter Logic Gaming", "Team Liquid", "FaZe Clan",
    "Natus Vincere", "Gambit Gaming", "FlipSid3 Tactics", "mousesports", "SK Gaming"
]
tournaments["Cologne 2015"] = [
    "Ninjas in Pyjamas", "Titan", "Luminosity Gaming", "Cloud9", "Team EnVyUs", "Virtus.pro",
    "Team SoloMid", "Fnatic", "Counter Logic Gaming", "Team Immunity", "Team Kinguin",
    "Natus Vincere", "Team eBettle", "FlipSid3 Tactics", "mousesports", "Renegades"
]
tournaments["Cluj-Napoca 2015"] = [
    "Ninjas in Pyjamas", "G2 Esports", "Cloud9", "Team Dignitas", "Team EnVyUs", "Virtus.pro",
    "Team SoloMid", "Fnatic", "Counter Logic Gaming", "Team Liquid", "Titan",
    "Natus Vincere", "Vexed Gaming", "FlipSid3 Tactics", "mousesports", "Luminosity Gaming"
]
tournaments["Katowice 2015"] = [
    "Ninjas in Pyjamas", "PENTA Sports", "Cloud9", "HellRaisers", "Team EnVyUs", "Virtus.pro",
    "Team SoloMid", "Fnatic", "Counter Logic Gaming", "LGB eSports", "Titan",
    "Natus Vincere", "3DMAX", "FlipSid3 Tactics", "Vox Eminor", "Keyd Stars"
]
tournaments["Cologne 2014"] = [
    "Ninjas in Pyjamas", "Team Dignitas", "Cloud9", "HellRaisers", "Team LDLC.com", "Virtus.pro",
    "London Conspiracy", "Fnatic", "dAT Team", "Epsilon eSports", "Titan",
    "Natus Vincere", "Copenhagen Wolves", "Team iBUYPOWER", "Vox Eminor", "MTS GameGod Wolf"
]

In [5]:
# imports
import requests
import urllib
import urllib3
import bs4
import json
import time
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Cipher import PKCS1_v1_5
import base64
from IPython.display import Image

In [6]:
# logging into steam
session = requests.session()  # using a session to maintain login status


# first, get the RSA key from Steam
username = "CalHacks_TestAccount"
password = b"CalHacks"
rsa_key_url = 'https://steamcommunity.com/login/getrsakey/'
get_rsa_payload = {
    'username': username,
    'donotcache': str(int(time.time() * 1000))
}

get_rsa_response = session.post(rsa_key_url, data=get_rsa_payload)
rsa_key = json.loads(get_rsa_response.text)
if rsa_key["success"]:
    print("Successfully retrieved RSA key")
# RSA public key has now been retrieved


# Use public key to encrypt password
mod = int(str(rsa_key["publickey_mod"]), 16)
exp = int(str(rsa_key["publickey_exp"]), 16)
key = RSA.construct((mod, exp))
cipher = PKCS1_v1_5.new(key)
encrypted_password = base64.b64encode(cipher.encrypt(password))


# Use encrypted password to login. Docs: https://bitbucket.org/Aerizeon/steamweb
login_url = 'https://steamcommunity.com/login/dologin/'
login_payload = {
    "password": encrypted_password,
    "username": username,
    "rsatimestamp": rsa_key["timestamp"],
    "oauth_client_id": "DE45CD61",
    "donotcache": str(int(time.time() * 1000)),
}
login_response = session.post(login_url, data=login_payload)
login_data = json.loads(login_response.text)
if login_data["success"]:
    print("Successfully logged in")
else:
    if login_data["emailauth_needed"]:
        auth_code = input("Email authentication code needed: ")
        login_payload = {
            "password": encrypted_password,
            "username": username,
            "rsatimestamp": rsa_key["timestamp"],
            "oauth_client_id": "DE45CD61",
            "donotcache": str(int(time.time() * 1000)),
            "emailauth": auth_code
        }
        login_response = session.post(login_url, data=login_payload)
        login_data = json.loads(login_response.text)
    else:
        print("Unexpected error with login, printing response")
        print(login_data)

Successfully retrieved RSA key
Successfully logged in


In [20]:
price_data = session.get(
    ("https://steamcommunity.com/market/pricehistory/?country=US&currency=3&appid=730&market_hash_name="
     "Sticker%20%7C%20Cloud9%20%7C%20London%202018"))
price_data.json()

'Sticker%20%7C%20Cloud9%20%7C%20London%202018'

In [32]:
basic_url = \
    "https://steamcommunity.com/market/pricehistory/?country=US&currency=3&appid=730&market_hash_name="


def store_data(tournament_name, team_name):
    hash_name = " | ".join(["Sticker", team_name, tournament_name])
    hash_name = hash_name.replace(" ", "%20").replace("|", "%7C")
    response_json = session.get(basic_url+hash_name).json()

    if not response_json['success']:
        print("Error in grabbing data for", tournament_name, team_name)
        return
    else:
        # successfully grabbed data, proceed with writing into csv
        

Error in grabbing data for lala baba


# LSTM

https://www.kaggle.com/amarpreetsingh/stock-prediction-lstm-using-keras?fbclid=IwAR2nYg-1dom0KmYCU89hNTgvlzGbFI4C3r531VoFMCG-SP8TXojz9jo9f2E

https://github.com/jaungiers/LSTM-Neural-Network-for-Time-Series-Prediction?fbclid=IwAR2nYg-1dom0KmYCU89hNTgvlzGbFI4C3r531VoFMCG-SP8TXojz9jo9f2E